In [1]:
#from kalmantv.cython import KalmanTV
from kalmantv.kalmantv_blas import KalmanTV
from KalmanTV import KalmanTV as KTV_py
from pykalman import standard as pks
import time
import numpy as np

In [2]:
%load_ext Cython

In [3]:
def rand_vec(n):
    """Generate a random vector."""
    return np.random.randn(n)

def rand_mat(n, p=None, pd=True):
    """Generate a random matrix, positive definite if `pd = True`."""
    if p is None:
        p = n
    V = np.zeros((n, p), order='F')
    V[:] = np.random.randn(n, p)
    if (p == n) & pd:
        V[:] = np.matmul(V, V.T)
    return V

In [4]:
def filter_time(n_state, n_meas):
    mu_state_past = rand_vec(n_state)
    var_state_past = rand_mat(n_state)
    mu_state = rand_vec(n_state)
    wgt_state = rand_mat(n_state, pd=False)
    var_state = rand_mat(n_state)
    x_meas = rand_vec(n_meas)
    mu_meas = rand_vec(n_meas)
    wgt_meas = rand_mat(n_meas, n_state, pd=False)
    var_meas = rand_mat(n_meas)
    loops = max(1, int(10000/n_state))
    start = time.time()
    for i in range(loops):
        mu_state_pred, var_state_pred = (
            pks._filter_predict(
                wgt_state, var_state,
                mu_state, mu_state_past,
                var_state_past
                )
        )
        _, mu_state_filt, var_state_filt = (
            pks._filter_correct(
                wgt_meas, var_meas,
                mu_meas, mu_state_pred,
                var_state_pred, x_meas
                )
            )
    end = time.time()
    pyk_time = (end - start)/loops
    
    ktv = KalmanTV(n_meas, n_state)
    mu_state_pred2 = np.empty(n_state)
    var_state_pred2 = np.empty((n_state, n_state), order='F')
    mu_state_filt2 = np.empty(n_state)
    var_state_filt2 = np.empty((n_state, n_state), order='F')
    start = time.time()
    for i in range(loops):
        ktv.filter(mu_state_pred2, var_state_pred2,
               mu_state_filt2, var_state_filt2,
               mu_state_past, var_state_past,
               mu_state, wgt_state, var_state,
               x_meas, mu_meas, wgt_meas, var_meas)
    end = time.time()
    ktv_time = (end - start)/loops
    return pyk_time, ktv_time

In [5]:
n_meas = 100
n_state = 2000

In [6]:
filter_time(n_state, n_meas)

(0.5403236865997314, 0.31964821815490724)

In [7]:
n_meas = 10
n_state = 200
filter_time(n_state, n_meas)

(0.0014026260375976562, 0.0005384969711303711)